# Introduction to Sentence Transformers

[Sentence Transformers](https://www.sbert.net/index.html) is a Python framework for sentence, text, and image embeddings based on the Sentence-BERT [paper](https://arxiv.org/abs/1908.10084).

## Imports & Inits

In [ ]:
%load_ext autoreload
%autoreload 2
%config IPCompleter.greedy=True

import pdb, pickle, sys, warnings, tqdm, time, torch, json, gzip
warnings.filterwarnings(action='ignore')

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from rank_bm25 import BM25Okapi
from sklearn.feature_extraction import _stop_words
import string

import os
import bokeh, bokeh.models, bokeh.plotting
import tensorflow.compat.v2 as tf
from simpleneighbors import SimpleNeighbors
from tqdm import tqdm, trange

from tqdm._tqdm_notebook import tqdm_notebook
tqdm_notebook.pandas()

from sentence_transformers import SentenceTransformer, util, CrossEncoder
device = 'cuda' if torch.cuda.is_available() else 'cpu'

We first load a pre-trained model. For this tutorial we will use the `all-mpnet-base-v2` model which was trained on all available training data (more than 1 billion training pairs) and is designed as a general purpose model. `sentence_transformers` have other pretrained models that can be found [here](https://www.sbert.net/docs/pretrained_models.html)

In [ ]:
model = SentenceTransformer('all-mpnet-base-v2')

## Functions

In [ ]:
def bm25_tokenizer(text):
  tokenized_doc = []
  for token in text.lower().split():
    token = token.strip(string.punctuation)

    if len(token) > 0 and token not in _stop_words.ENGLISH_STOP_WORDS:
      tokenized_doc.append(token)
  return tokenized_doc

def search(query):
  print("Input question:", query)

  ##### BM25 search (lexical search) #####
  bm25_scores = bm25.get_scores(bm25_tokenizer(query))
  top_n = np.argpartition(bm25_scores, -5)[-5:]
  bm25_hits = [{'corpus_id': idx, 'score': bm25_scores[idx]} for idx in top_n]
  bm25_hits = sorted(bm25_hits, key=lambda x: x['score'], reverse=True)

  print("Top-3 lexical search (BM25) hits")
  for hit in bm25_hits[0:3]:
    print("\t{:.3f}\t{}".format(hit['score'], passages[hit['corpus_id']].replace("\n", " ")))

  ##### Sematic Search #####
  # Encode the query using the bi-encoder and find potentially relevant passages
  question_embedding = bi_encoder.encode(query, convert_to_tensor=True)
  question_embedding = question_embedding.cuda()
  hits = util.semantic_search(question_embedding, corpus_embeddings, top_k=top_k)
  hits = hits[0]  # Get the hits for the first query

  ##### Re-Ranking #####
  # Now, score all retrieved passages with the cross_encoder
  cross_inp = [[query, passages[hit['corpus_id']]] for hit in hits]
  cross_scores = cross_encoder.predict(cross_inp)

  # Sort results by the cross-encoder scores
  for idx in range(len(cross_scores)):
    hits[idx]['cross-score'] = cross_scores[idx]

  # Output of top-5 hits from bi-encoder
  print("\n-------------------------\n")
  print("Top-3 Bi-Encoder Retrieval hits")
  hits = sorted(hits, key=lambda x: x['score'], reverse=True)
  for hit in hits[0:3]:
    print("\t{:.3f}\t{}".format(hit['score'], passages[hit['corpus_id']].replace("\n", " ")))

  # Output of top-5 hits from re-ranker
  print("\n-------------------------\n")
  print("Top-3 Cross-Encoder Re-ranker hits")
  hits = sorted(hits, key=lambda x: x['cross-score'], reverse=True)
  for hit in hits[0:3]:
    print("\t{:.3f}\t{}".format(hit['cross-score'], passages[hit['corpus_id']].replace("\n", " ")))

## Semantic Textual Similarity

Sentence embeddings can be used to compare how semantically similar two sentences are. A common way to compare semantic content is to use `cosine` similarity.

In [ ]:
sentences = [
  'The cat sits outside',
  'A man is playing guitar',
  'I love pasta',
  'The new movie is awesome',
  'The cat plays in the garden',
  'A woman watches TV',
  'The new movie is so great',
  'Do you like pizza?',
]

embeddings = model.encode(sentences, convert_to_tensor=True)
cosine_scores = util.cos_sim(embeddings, embeddings)

#Find the pairs with the highest cosine similarity scores
pairs = []
for i in range(len(cosine_scores)-1):
  for j in range(i+1, len(cosine_scores)):
    pairs.append({'index': [i, j], 'score': cosine_scores[i][j].cpu().numpy()})

#Sort scores in decreasing order
pairs = sorted(pairs, key=lambda x: x['score'], reverse=True)

sim = {'Sentence 1': [], 'Sentence 2': [], 'Score': []}
for pair in pairs[0:10]:
  i, j = pair['index']
  sim['Sentence 1'].append(sentences[i])
  sim['Sentence 2'].append(sentences[j])
  sim['Score'].append(pair['score'])

pd.DataFrame(sim)

## Semantic Search

Semantic search improves search accuracy by understanding the content of the search query. In addition to finding documents based on lexical matches, semantic search can also synonyms.

In [ ]:
corpus = [
  'A man is eating food.',
  'A man is eating a piece of bread.',
  'The girl is carrying a baby.',
  'A man is riding a horse.',
  'A woman is playing violin.',
  'Two men pushed carts through the woods.',
  'A man is riding a white horse on an enclosed ground.',
  'A monkey is playing drums.',
  'A cheetah is running behind its prey.'
]
corpus_embeddings = model.encode(corpus, convert_to_tensor=True)

# Query sentences:
queries = [
  'A man is eating pasta.',
  'Someone in a gorilla costume is playing a set of drums.',
  'A cheetah chases prey on across a field.'
]


# Find the closest 5 sentences of the corpus for each query sentence based on cosine similarity
top_k = min(5, len(corpus))
for query in queries:
  query_embedding = model.encode(query, convert_to_tensor=True)

  # We use cosine-similarity and torch.topk to find the highest 5 scores
  cos_scores = util.cos_sim(query_embedding, corpus_embeddings)[0]
  top_results = torch.topk(cos_scores, k=top_k)

  print("\n\n======================\n\n")
  print("Query:", query)
#   print("\nTop 5 most similar sentences in corpus:")
  
  sim = {'Similar Sentence': [], 'Similarity Score': []}  
  for score, idx in zip(top_results[0], top_results[1]):
    sim['Similar Sentence'].append(corpus[idx])
    sim['Similarity Score'].append(score.cpu().numpy())
  
  display(pd.DataFrame(sim))

#         print(corpus[idx], "(Score: {:.4f})".format(score))

## Paraphrase Mining

Paraphrase mining is the task of finding texts with identical or similar meaning in a large corpus of sentences.

In [ ]:
# Single list of sentences - Possible tens of thousands of sentences
sentences = [
  'The cat sits outside',
  'A man is playing guitar',
  'I love pasta',
  'The new movie is awesome',
  'The cat plays in the garden',
  'A woman watches TV',
  'The new movie is so great',
  'Do you like pizza?'
]

paraphrases = util.paraphrase_mining(model, sentences)
sim = {'Source Sentence': [], 'Paraphrase': [], 'Similarity Score': []}
for paraphrase in paraphrases[0:10]:
    score, i, j = paraphrase
    sim['Source Sentence'].append(sentences[i])
    sim['Paraphrase'].append(sentences[j])
    sim['Similarity Score'].append(score)
    
pd.DataFrame(sim)    

## Retrieval & Re-ranking

Given a search query, a **retrieval system** retrieves a large list of potentially relevant hits for the query using a **bi-encoder**. Then a **re-ranker** based on a **cross-encoder** scores the relevancy of all candidates for the given search query.

The dataset we use here is the smaller Simple English Wikipedia as document collection to provide answers to user questions and search queries can be downloaded [here](http://sbert.net/datasets/simplewiki-2020-11-01.jsonl.gz).

In [ ]:
bi_encoder = SentenceTransformer('multi-qa-MiniLM-L6-cos-v1')
bi_encoder.max_seq_length = 256     #Truncate long passages to 256 tokens
top_k = 32                          #Number of passages we want to retrieve with the bi-encoder

cross_encoder = CrossEncoder('cross-encoder/ms-marco-MiniLM-L-6-v2')
wikipedia_filepath = '../project_dir/simplewiki-2020-11-01.jsonl.gz'
passages = []
with gzip.open(wikipedia_filepath, 'rt', encoding='utf8') as fIn:
  for line in fIn:
    data = json.loads(line.strip())
    #passages.extend(data['paragraphs'])
    passages.append(data['paragraphs'][0])

print("Passages:", len(passages))

In [ ]:
corpus_embeddings = bi_encoder.encode(passages, convert_to_tensor=True, show_progress_bar=True)

tokenized_corpus = []
for passage in tqdm_notebook(passages):
  tokenized_corpus.append(bm25_tokenizer(passage))

bm25 = BM25Okapi(tokenized_corpus)

queries = [
  'What is the best orchestra in the world?',
  'Number countries Europe',
  'When did the cold war end?',
  'How long do cats live?',
  'How many people live in Toronto?',
  'Oldest US president',
  'Coldest place earth',
  'Elon Musk year birth',
  'Paris eiffel tower',
  'Which US president was killed?',
  'When is Chinese New Year',  
]

In [ ]:
for query in queries:
  print(search(query))
  print("\n\n======================\n\n")

## Computing Sentence Embeddings

Sentence embeddings compute an embedding vector for a piece of text. While thinking sentences is natural here, this method can also be used on shorter phrases and longer text containing multiple sentences.

In [ ]:
sentences = [
  'This framework generates embeddings for each input sentence',
  'Sentences are passed as a list of string.',
  'The quick brown fox jumps over the lazy dog.'
]

embeddings = model.encode(sentences)
for sentence, embedding in zip(sentences, embeddings):
  print("Sentence:", sentence)
  print("Embedding:", embedding)
  print("")

## Cross-lingual Retrieval 

We show a use case of sentence transformer model in cross-lingual retrieval: cross-lingual semantic-similarity search engine

This notebook has been adapted from [this paper](https://aclanthology.org/2020.acl-demos.12/).

### Load Transformer Model

We load `xlm-mlm-100-1280` transformer which is language model trained for masked token prediction task on a dataset with 100 languages.

In [ ]:
model = SentenceTransformer('xlm-mlm-100-1280')

def embed_text(input):
  return model.encode(input)

### Calculate Multilingual Semantic Similarity

With sentence embeddings in hand, we can take their dot-product to measure their semantically similarity.

In [ ]:
# an example of calculating semantic similarity between two multilingual sentences
sentence1 = "natural language understanding" #en
sentence2 = "comprensión del lenguaje natural" #es
# encode sentences to get their embeddings
embedding1 = model.encode(sentence1, convert_to_tensor=True)
embedding2 = model.encode(sentence2, convert_to_tensor=True)
# compute similarity scores of two embeddings
cosine_scores = util.pytorch_cos_sim(embedding1, embedding2)
print("Sentence 1:", sentence1)
print("Sentence 2:", sentence2)
print("Similarity score:", cosine_scores.item())


# Retrieve top K most similar sentences from a multilingual corpus given a query sentence
corpus = ["I like Python because I can build AI applications",
          "Me gusta Python porque puedo hacer análisis de datos",
          "The cat sits on the ground",
         "El gato camina por la acera."]
# encode corpus to get corpus embeddings
corpus_embeddings = model.encode(corpus, convert_to_tensor=True)
sentence = "I like Javascript because I can build web applications"
# encode sentence to get sentence embeddings
sentence_embedding = model.encode(sentence, convert_to_tensor=True)
# top_k results to return
top_k=2
# compute similarity scores of the sentence with the corpus
cos_scores = util.pytorch_cos_sim(sentence_embedding, corpus_embeddings)[0]
# Sort the results in decreasing order and get the first top_k
top_results = np.argpartition(-cos_scores.cpu(), range(top_k))[0:top_k]
print("\nSentence:", sentence)
print("Top", top_k, "most similar sentences in corpus:")
for idx in top_results[0:top_k]:
    print(corpus[idx], "(Score: %.4f)" % (cos_scores[idx]))


### Visualize Multilingual Semantic Similarity

With sentence embeddings in hand, we can take their dot-product to visualize how similar sentences are between languages. A darker color indicates the embeddings are semantically similar.

In [ ]:
def visualize_similarity(embeddings_1, embeddings_2, labels_1, labels_2,
                         plot_title,
                         plot_width=1200, plot_height=600,
                         xaxis_font_size='12pt', yaxis_font_size='12pt'):

  assert len(embeddings_1) == len(labels_1)
  assert len(embeddings_2) == len(labels_2)

  # cosine based text similarity
  sim = util.pytorch_cos_sim(embeddings_1, embeddings_2)

  embeddings_1_col, embeddings_2_col, sim_col = [], [], []
  for i in range(len(embeddings_1)):
    for j in range(len(embeddings_2)):
      embeddings_1_col.append(labels_1[i])
      embeddings_2_col.append(labels_2[j])
      sim_col.append(np.float(sim[i][j]))
  df = pd.DataFrame(zip(embeddings_1_col, embeddings_2_col, sim_col),
                    columns=['embeddings_1', 'embeddings_2', 'sim'])

  mapper = bokeh.models.LinearColorMapper(
      palette=[*reversed(bokeh.palettes.YlOrRd[9])], low=df.sim.min(),
      high=df.sim.max())

  p = bokeh.plotting.figure(title=plot_title, x_range=labels_1,
                            x_axis_location="above",
                            y_range=[*reversed(labels_2)],
                            plot_width=plot_width, plot_height=plot_height,
                            tools="save",toolbar_location='below', tooltips=[
                                ('pair', '@embeddings_1 ||| @embeddings_2'),
                                ('sim', '@sim')])
  p.rect(x="embeddings_1", y="embeddings_2", width=1, height=1, source=df,
         fill_color={'field': 'sim', 'transform': mapper}, line_color=None)

  p.title.text_font_size = '12pt'
  p.axis.axis_line_color = None
  p.axis.major_tick_line_color = None
  p.axis.major_label_standoff = 16
  p.xaxis.major_label_text_font_size = xaxis_font_size
  p.xaxis.major_label_orientation = 0.25 * np.pi
  p.yaxis.major_label_text_font_size = yaxis_font_size
  p.min_border_right = 300

  bokeh.io.output_notebook()
  bokeh.io.show(p)


In [ ]:
# Some texts of different lengths in different languages.
english_sentences = ['dog', 'Puppies are nice.', 'I enjoy taking long walks along the beach with my dog.']
spanish_sentences = ['perro', 'Los cachorros son agradables.', 'Disfruto de dar largos paseos por la playa con mi perro.']

# Compute embeddings.
en_result = embed_text(english_sentences)
es_result = embed_text(spanish_sentences)

# visualize semantic similarity
visualize_similarity(en_result, es_result, english_sentences, spanish_sentences, 'English-Spanish Similarity')

### Build a simple cross-lingual document retrieval engine

#### Download Data to Index
Download news sentences in multiples languages (English and Spanish) from the [News Commentary Corpus](http://opus.nlpl.eu/News-Commentary-v11.php) [[1]](http://citeseerx.ist.psu.edu/viewdoc/download?doi=10.1.1.673.2874&rep=rep1&type=pdf).

To speed up the demo, we limit to 1000 sentences per language.


In [ ]:
def download_files(corpus_metadata):
  language_to_sentences = {}
  language_to_news_path = {}
  for language_code, zip_file, news_file, language_name in corpus_metadata:
    zip_path = tf.keras.utils.get_file(
        fname=zip_file,
        origin='http://opus.nlpl.eu/download.php?f=News-Commentary/v11/moses/' + zip_file,
        extract=True)
    news_path = os.path.join(os.path.dirname(zip_path), news_file)
    language_to_sentences[language_code] = pd.read_csv(news_path, sep='\t', header=None)[0][:1000]
    language_to_news_path[language_code] = news_path

    print('{:,} {} sentences'.format(len(language_to_sentences[language_code]), language_name))
  return language_to_sentences, language_to_news_path


In [ ]:
corpus_metadata = [
    ('en', 'en-es.txt.zip', 'News-Commentary.en-es.en', 'English'),
    ('es', 'en-es.txt.zip', 'News-Commentary.en-es.es', 'Spanish'),
]
language_to_sentences, language_to_news_path = download_files(corpus_metadata)

#### Get document embeddings 

In [ ]:
sample_size = 1000
language_to_embeddings = {}
for language_code, zip_file, news_file, language_name in corpus_metadata:
  print('\nComputing {} embeddings'.format(language_name))
  with tqdm(total=len(language_to_sentences[language_code])) as pbar:
    df = pd.read_csv(language_to_news_path[language_code],
                             sep='\t',header=None).head(sample_size)
    for i in range(len(df)):
      language_to_embeddings.setdefault(language_code, []).append(embed_text(df[0][i]))


#### Building an index of semantic vectors

Use the [SimpleNeighbors](https://pypi.org/project/simpleneighbors/) library---which is a wrapper for the [Annoy](https://github.com/spotify/annoy) library---to efficiently look up results from the corpus.

In [ ]:
%%time

num_index_trees = 40
language_name_to_index = {}
embedding_dimensions = len(list(language_to_embeddings.values())[0][0])
for language_code, zip_file, news_file, language_name in corpus_metadata:
  print('\nAdding {} embeddings to index'.format(language_name))
  index = SimpleNeighbors(embedding_dimensions, metric='cosine')

  for i in trange(len(language_to_sentences[language_code])):
    index.add_one(language_to_sentences[language_code][i], language_to_embeddings[language_code][i])

  print('Building {} index with {} trees...'.format(language_name, num_index_trees))
  index.build(n=num_index_trees)
  language_name_to_index[language_name] = index

In [ ]:
%%time

num_index_trees = 60
print('Computing mixed-language index')
combined_index = SimpleNeighbors(embedding_dimensions, metric='cosine')
for language_code, zip_file, news_file, language_name in corpus_metadata:
  print('Adding {} embeddings to mixed-language index'.format(language_name))
  for i in trange(len(language_to_sentences[language_code])):
    annotated_sentence = '({}) {}'.format(language_name, language_to_sentences[language_code][i])
    combined_index.add_one(annotated_sentence, language_to_embeddings[language_code][i])

print('Building mixed-language index with {} trees...'.format(num_index_trees))
combined_index.build(n=num_index_trees)

#### Validate the document retrieval pipeline

1.   Retrieve sentences from the corpus that are semantically similar to the given query.
2.   Cross-lingual: Issue queries in a distinct language than the indexed corpus
3.   Mixed-corpus: Issue queries in one language and retrieve similar documents from multiple languages

##### Cross-lingual

Issue queries in a distinct language than the indexed corpus.

**TODO Display sents with scores in DF**

In [ ]:
sample_query = 'The stock market fell four points.'
index_language = 'Spanish'  #@param ["English", "Spanish"]
num_results = 10  #@param {type:"slider", min:0, max:100, step:10}

query_embedding = embed_text(sample_query)
search_results = language_name_to_index[index_language].nearest(query_embedding, n=num_results)

print('{} sentences similar to: "{}"\n'.format(index_language, sample_query))
search_results

In [ ]:
sample_query = 'Desde entonces, el índice ha trepado por encima de 10.000.'
index_language = 'English'  #@param ["English", "Spanish"]
num_results = 10  #@param {type:"slider", min:0, max:100, step:10}

query_embedding = embed_text(sample_query)
search_results = language_name_to_index[index_language].nearest(query_embedding, n=num_results)

print('{} sentences similar to: "{}"\n'.format(index_language, sample_query))
search_results

##### Mixed-corpus capabilities

Issue a query in English and the results will come from the any of the indexed languages.

In [ ]:
query = 'The stock market fell four points.'  #@param {type:"string"}
num_results = 10  #@param {type:"slider", min:0, max:100, step:10}

query_embedding = embed_text(query)
search_results = combined_index.nearest(query_embedding, n=num_results)

print('Multilingual sentences similar to: "{}"\n'.format(query))
search_results